# Manuactured Variables

## Performing Sentiment Analysis on Description, Tagline, and Overview

In [1]:
from transformers import pipeline

classifier = pipeline(
    "sentiment-analysis", 
    model="distilbert-base-uncased-finetuned-sst-2-english",
    truncation=True
    )

Device set to use cpu

In [2]:
import pandas as pd
import numpy as np

movies_df = pd.read_csv("../data/output_data/clean_movies.csv")

#Description Sentiment Analysis
movies_df["Descr_Sentiment"] = movies_df["Description"].apply(lambda x: classifier(x)[0]["label"])

In [3]:
# Overview Sentiment Analysis
movies_df["Tagline_Sentiment"] = movies_df["Tagline"].apply(lambda x: classifier(x)[0]["label"])

In [4]:
# converting the neg/pos to label 0/1
movies_df["Descr_Sentiment"] = movies_df["Descr_Sentiment"].replace({"NEGATIVE": 0, "POSITIVE": 1})
movies_df["Tagline_Sentiment"] = movies_df["Tagline_Sentiment"].replace({"NEGATIVE": 0, "POSITIVE": 1})

/var/folders/f7/px3ynpyn7l3g5j59yyvbggrw0000gn/T/ipykernel_41943/2660690932.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  movies_df["Descr_Sentiment"] = movies_df["Descr_Sentiment"].replace({"NEGATIVE": 0, "POSITIVE": 1})
/var/folders/f7/px3ynpyn7l3g5j59yyvbggrw0000gn/T/ipykernel_41943/2660690932.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  movies_df["Tagline_Sentiment"] = movies_df["Tagline_Sentiment"].replace({"NEGATIVE": 0, "POSITIVE": 1})

In [5]:
# dropping the original columns
movies_df = movies_df.drop(columns=["Tagline", "Description"])

In [6]:
movies_df.to_csv("../data/output_data/clean_movies_sent.csv", index=False)

## Creating the Starpower Variable

In [7]:
import pandas as pd
import numpy as np

movies_df = pd.read_csv("../data/output_data/clean_movies_sent_fem.csv")

In [8]:
# Loading in the list of A-list and B-list actors/actresses 
a_list_actors_df = pd.read_csv("../data/input_data/a_list_actors.csv")
a_list_actresses_df = pd.read_csv("../data/input_data/a_list_actresses.csv")
b_list_df = pd.read_csv("../data/input_data/b_list.csv")

# taking just the names from the dataframes
a_list_actors = a_list_actors_df['Name'].tolist()
a_list_actors = pd.DataFrame(a_list_actors, columns=['Name'])

a_list_actresses = a_list_actresses_df['Name'].tolist()
a_list_actresses = pd.DataFrame(a_list_actresses, columns=['Name'])

b_list = b_list_df['Name'].tolist()
b_list = pd.DataFrame(b_list, columns=['Name'])

# combining the A-list actors and actresses into one dataframe
a_list = pd.concat([a_list_actors, a_list_actresses], ignore_index=True)

print(a_list.head(5))
print(b_list.head(5))

num_a_list = len(a_list)
num_b_list = len(b_list)

print(f'Number of A-listers: {num_a_list}')
print(f'Number of b-listers: {num_b_list}')

              Name
0        Al Pacino
1  Tommy Lee Jones
2    Harrison Ford
3   Robert De Niro
4    Kevin Costner
               Name
0      Ansel Elgort
1       Wes Bentley
2  Chiwetel Ejiofor
3        Karl Urban
4   Britt Robertson
Number of A-listers: 200
Number of b-listers: 376

In [9]:
# Creating the variable to calculate the starpower metric

# splitting to get all four actors/actresses into separate columns
if movies_df['Actors'].notna().any():
    actors_split = movies_df['Actors'].str.split(',', expand=True)
    movies_df['actor1'] = actors_split[0].str.strip()
    movies_df['actor2'] = actors_split[1].str.strip()
    movies_df['actor3'] = actors_split[2].str.strip()
else:
    print("The 'Actors' column is empty or missing.")

# creating the metric column
movies_df['starpower'] = 0

# Check if the first actor is in the a-list
movies_df.loc[movies_df['actor1'].isin(a_list['Name'].values), 'starpower'] += 2
movies_df.loc[movies_df['actor2'].isin(a_list['Name'].values), 'starpower'] += 2
movies_df.loc[movies_df['actor3'].isin(a_list['Name'].values), 'starpower'] += 2

movies_df.loc[movies_df['actor1'].isin(b_list['Name'].values), 'starpower'] += 1
movies_df.loc[movies_df['actor2'].isin(b_list['Name'].values), 'starpower'] += 1
movies_df.loc[movies_df['actor3'].isin(b_list['Name'].values), 'starpower'] += 1

# making it an average value
movies_df['starpower'] = movies_df['starpower'] / 3

# checking the new variable
print(movies_df[['actor1', 'actor2', 'actor3', 'starpower']].head(10))

# dropping the actor columns
movies_df = movies_df.drop(columns=["actor1", "actor2", "actor3", "Actors"])

           actor1             actor2           actor3  starpower
0   Mark Wahlberg      Tyrese Gibson       André 3000   0.666667
1      Jamie Bell        Andy Serkis     Daniel Craig   1.333333
2   Ryan Reynolds       Blake Lively  Peter Sarsgaard   0.333333
3     Marc Singer      Tanya Roberts         Rip Torn   0.000000
4  Tom Hiddleston  Samuel L. Jackson      Brie Larson   1.000000
5   Jeremy Renner           Ed Helms     Jake Johnson   0.000000
6   Frankie Muniz       Amanda Bynes    Paul Giamatti   1.000000
7      Ben Barnes     Skandar Keynes   Georgie Henley   0.000000
8   Jason Bateman        Charlie Day   Jason Sudeikis   1.000000
9      Jack Black  Ana de la Reguera   Héctor Jiménez   0.333333

In [10]:
movies_df.to_csv("../data/output_data/clean_movies_sent_fem_star.csv", index=False)

## Creating the Female Actress Lead Variable

In [11]:
import pandas as pd
import numpy as np

movies_df = pd.read_csv("../data/output_data/clean_movies_sent.csv")

In [12]:
# checking how accurate the first listed actor/actress is in terms of being the star or main character
first_actors = movies_df['Actors'].str.split(',').str[0].str.strip()
print(first_actors.head(20))

print(movies_df['Title'].head(20))

0          Mark Wahlberg
1             Jamie Bell
2          Ryan Reynolds
3            Marc Singer
4         Tom Hiddleston
5          Jeremy Renner
6          Frankie Muniz
7             Ben Barnes
8          Jason Bateman
9             Jack Black
10            Chris Pine
11        Angelina Jolie
12          Sam Rockwell
13            Nick Nolte
14        Timothy Dalton
15             Al Pacino
16        Clint Eastwood
17    Natasha Henstridge
18           Hilary Duff
19       Jake Gyllenhaal
Name: Actors, dtype: object
0                                Four Brothers
1                     The Adventures of Tintin
2                                Green Lantern
3                              The Beastmaster
4                           Kong: Skull Island
5                                          Tag
6                                 Big Fat Liar
7     The Chronicles of Narnia: Prince Caspian
8                              Horrible Bosses
9                                  Nacho Libre
10

In [13]:
# getting the female actresses names from excel file

# opening the file
female_df = pd.read_excel("../data/input_data/female_actresses.xlsx")
female_df = female_df.rename(columns={'A': 'Name'})

# splitting to get the names only
female_df['Name'] = female_df['Name'].str.split(' born').str[0].str.split(r'\d').str[0].str.strip()

# deleting the rows that are just the letter headers
female_df = female_df[~female_df['Name'].isin(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'))]

num_actresses = len(female_df)
print(f'There are {num_actresses} actresses in the list')

There are 2452 actresses in the list

In [14]:
# Creating the variable to indicate if first actor is female
first_actors = movies_df['Actors'].str.split(',').str[0].str.strip()
movies_df['First_Actor'] = first_actors

# Check if the first actor is in the female name list
movies_df['Female_Lead'] = movies_df['First_Actor'].apply(lambda x: int(any(name in x for name in female_df['Name'])))

# checking the new variable
print(movies_df[['Title', 'First_Actor', 'Female_Lead']].head(10))
print(movies_df[['Title', 'First_Actor', 'Female_Lead']].tail(10))

# dropping the first_Actor column
movies_df = movies_df.drop(columns=["First_Actor"])

                                      Title     First_Actor  Female_Lead
0                             Four Brothers   Mark Wahlberg            0
1                  The Adventures of Tintin      Jamie Bell            0
2                             Green Lantern   Ryan Reynolds            0
3                           The Beastmaster     Marc Singer            0
4                        Kong: Skull Island  Tom Hiddleston            0
5                                       Tag   Jeremy Renner            0
6                              Big Fat Liar   Frankie Muniz            0
7  The Chronicles of Narnia: Prince Caspian      Ben Barnes            0
8                           Horrible Bosses   Jason Bateman            0
9                               Nacho Libre      Jack Black            0
              Title         First_Actor  Female_Lead
2806     The Family      Robert De Niro            0
2807      The Shack     Sam Worthington            0
2808  The Dead Zone  Christopher Walke

In [15]:
# Count of the split
female_lead_count = movies_df['Female_Lead'].value_counts()
print(female_lead_count)

Female_Lead
0    2328
1     488
Name: count, dtype: int64

In [16]:
movies_df.to_csv("../data/output_data/clean_movies_sent_fem.csv", index=False)